In [1]:
# Reads beam actor messages from the last create beam output dir
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.cluster.hierarchy as sch
import util.beam_data as bd
from util.data import find_last_created_dir
from util.data import meaningful

schema = {
    "type": str,
    "sender_parent": str,
    "sender_name": str,
    "receiver_parent": str,
    "receiver_name": str,
    "payload": str,
    "state": str,
    "tick": int,
    "triggerId": pd.Int64Dtype(),
}
root = "../beam_root"
# beam.debug.messageLogging = true (in beam.conf)

msg_file_num = 0
def load_data(dir, it):
    return pd.read_csv(f"{dir}/ITERS/it.{it}/{it}.actor_messages_{msg_file_num}.csv.gz", dtype=schema)
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# beam_out_path = "beamville/rh-reservation-before-walking__2023-02-09_18-36-57_bjq" #good
beam_out_path = "beamville/rh-stop-early-reservation-default-manager__2023-02-14_19-50-15_kzc"   #bad
beam_out_path = None
beam_out = find_last_created_dir(f"{root}/output", level = 1, num = 0) if beam_out_path is None else f"{root}/output/{beam_out_path}"
print(f"Using beam out dir: {beam_out}")

ifields = ['type', 'sender_name', 'receiver_name', 'payload', 'state', 'tick', 'triggerId']

msg = load_data(beam_out, 0)
# msg = load_data('enroute-ci-stuck__2022-02-01_11-07-10_pxv', 0)
msg.head()

Using beam out dir: ../output/sf-light/rhtp-develop-with-rh-stops__2023-10-16_12-48-56_fie


,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
0,message,NaN,BeamMobsim.iteration,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
1,message,NaN,BeamMobsim.iteration,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
2,message,NaN,BeamMobsim.iteration,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
3,message,NaN,BeamMobsim.iteration,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
4,message,NaN,BeamMobsim.iteration,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1


In [2]:
# A particular person messages
person_id = '012202-2013000609715-0-5899892'
display(f"-- Person sequence: {person_id} --")

trigger_ids = msg[((msg['sender_name'] == person_id) & (msg['sender_parent'] != "population")) | ((msg['receiver_name'] == person_id) & (msg['receiver_parent'] != "population"))
                  | (msg['payload'].str.contains(f"/{person_id}#"))
]['triggerId'].unique()
trigger_ids = [id for id in trigger_ids if id >= 0]

person4 = msg[
    (
            ((msg['sender_name'] == person_id) & (msg['sender_parent'] != "population"))
            | ((msg['receiver_name'] == person_id) & (msg['receiver_parent'] != "population"))
            | (msg['triggerId'].isin(trigger_ids))
            | (msg['payload'].str.contains(f"/{person_id}#"))
    )
    # & (msg['payload'].str.startswith('Routing'))
]

person4

'-- Person sequence: 012202-2013000609715-0-5899892 --'

,type,sender_parent,sender_name,receiver_parent,receiver_name,payload,state,tick,triggerId
7474,message,population,012202-2013000609715-0,BeamMobsim.iteration,scheduler,"ScheduleTrigger(InitializeTrigger(0),Actor[akk...",NaN,-1,-1
7644,event,BeamMobsim.iteration,scheduler,population/012202-2013000609715-0,012202-2013000609715-0-5899892,"TriggerWithId(InitializeTrigger(0),2592)","BasePersonData(0,None,List(),Vector(),None,Non...",0,2592
7646,transition,BeamMobsim.iteration,scheduler,population/012202-2013000609715-0,012202-2013000609715-0-5899892,Uninitialized,Initialized,-1,-94677838
18245,message,population/012202-2013000609715-0,012202-2013000609715-0-5899892,BeamMobsim.iteration,scheduler,"CompletionNotice(2592,Vector(ScheduleTrigger(A...",NaN,-1,2592
18276,event,BeamMobsim.iteration,scheduler,population/012202-2013000609715-0,012202-2013000609715-0-5899892,"TriggerWithId(ActivityStartTrigger(0),5604)","BasePersonData(0,None,List(),Vector(),None,Non...",0,5604
...,...,...,...,...,...,...,...,...,...
2599706,message,population/012202-2013000609715-0,012202-2013000609715-0-5899892,BeamMobsim.iteration,scheduler,"CompletionNotice(656649,Vector(ScheduleTrigger...",NaN,-1,656649
2634230,event,BeamMobsim.iteration,scheduler,population/012202-2013000609715-0,012202-2013000609715-0-5899892,"TriggerWithId(ActivityEndTrigger(67668),727549)","BasePersonData(2,None,List(),Vector(),None,Non...",67668,727549
2634276,message,population/012202-2013000609715-0,012202-2013000609715-0-5899892,BeamMobsim.iteration,scheduler,"CompletionNotice(727549,Vector())",NaN,-1,727549
3366986,event,population,012202-2013000609715-0,population/012202-2013000609715-0,012202-2013000609715-0-5899892,Finish,"BasePersonData(2,None,List(),Vector(),None,Non...",-1,-1
